In [ ]:
# !pip install ydata-profiling
# !pip install ipywidgets
# !pip install seaborn
# !pip install hvplot
# !pip install ipympl
# !pip install plotly
# !pip install jupyter_bokeh
# ! pip install shap
# !pip install dash dash-bootstrap-components
# !pip install dask
# !pip install pyarrow

In [ ]:
%cd ..

In [ ]:
data_path = "data/Quote_and_Order_Data_2025_01_30.csv"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
import hvplot.pandas
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import shap

hvplot.extension('plotly')
output_notebook()
# %matplotlib widget
%matplotlib ipympl
%matplotlib inline
plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')
shap.initjs()

In [ ]:
from gomat_markup_opt.core.train_conversion_probability_model import _prepare_input_data

In [ ]:
data = pd.read_csv(
    data_path,
    delimiter=",",
    encoding="latin1",
    on_bad_lines="warn",
    engine="python",
)

In [ ]:
train_data = _prepare_input_data(data)

In [ ]:
train_data.head()

In [ ]:
train_data.columns

In [ ]:
from gomat_markup_opt.core.train_conversion_probability_model import (
    TOP_10_REGIONS,
    CUSTOMER_COLUMN,
    ORDER_DATE,
    ORDER_ID_COLUMN,
    PRODUCT_ID_COLUMN,
    PLANT_CATEGORY_COLUMN,
    PLANT_NAME_COLUMN,
    QUOTE_STATUS,
    REGION_COLUMN,
    SELLER_PRICE,
    SELLER_PRICE_RELATIVE_TO_MARKET,
    SELLER_TOTAL,
    SELECTED_COLUMNS,
    MARKUP_COLUMN,
    TOTAL_ITEMS_COLUMN,
    REQUEST_PURPOSE_COLUMN,
    QTO_COLUMN,
)

# Build markup search space

In [ ]:
SELLER_PRICE_BINS = "Seller Total Bins"

In [ ]:
PLANT_REGION_COLS = [
    REGION_COLUMN,
    PLANT_NAME_COLUMN,
]

In [ ]:
GROUP_BY_COLS = [
        QTO_COLUMN,
        REQUEST_PURPOSE_COLUMN,
        REGION_COLUMN,

        PLANT_NAME_COLUMN, 
        SELLER_PRICE_BINS
        ]

In [ ]:
THRESHOLD_DT = "2024-01-01"
def filter_items_by_last_dt(data):
    last_buy_date = data.groupby(PLANT_NAME_COLUMN)[ORDER_DATE].apply(lambda x: pd.to_datetime(x).max())
    last_buy_date.name = "last_buy_date"
    data = data.merge(last_buy_date, on=PLANT_NAME_COLUMN)
    data = data[data["last_buy_date"] >= pd.Timestamp(THRESHOLD_DT)]
    return data

def filter_by_num_sales(data, minimum_sales= 5):
    more_than_five_sales_data = data[PLANT_REGION_COLS].value_counts()
    more_than_five_sales_index = more_than_five_sales_data[more_than_five_sales_data > minimum_sales].index.tolist()

    filtered_data = data.set_index(PLANT_REGION_COLS).loc[more_than_five_sales_index].reset_index()
    return filtered_data

In [ ]:
BINS = 4

In [ ]:
df = filter_by_num_sales(train_data)
df = filter_items_by_last_dt(df)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class QuantileBinClassifier(BaseEstimator, TransformerMixin):
    def __init__(self, target_column=SELLER_PRICE, group_by_cols=PLANT_REGION_COLS, n_bins=BINS, quantile_feature_colname=SELLER_PRICE_BINS):
        """
        n_bins: number of bins for each category
        """
        self.target_column = target_column
        self.group_by_cols = group_by_cols
        self.n_bins = n_bins
        self.quantile_feature_colname = quantile_feature_colname
        self.per_group_bins_: pd.Series
        self.is_fitted_ : bool = False

    def fit(self, X, y=None):
        """
        Register bins for each groupby category.
        """
        def _get_bin_edges(df):
            group_values = df[self.target_column]
            _, bin_edges = pd.qcut(group_values, q=self.n_bins, retbins=True, duplicates='drop')
            return bin_edges
            
        self.per_group_bins_ = X.groupby(self.group_by_cols).apply(_get_bin_edges, include_groups=False)
        self.is_fitted_ = True
        return self
    
    def fit_transform(self, X: pd.DataFrame, y=None):
        self.fit(X)
        return self.transform(X)    
    
    def transform(self, X):
        """
        Predict quantile classes for each groupby category.
        """
        def _predict_single(row):
            group = tuple(row.loc[self.group_by_cols])
            try:
                bin_edges = self.per_group_bins_.loc[group]
            except KeyError:
                return np.nan
            return np.digitize(row.loc[self.target_column], bin_edges[1:-1], right=True)

        processed_df = X.copy(deep=True)
        processed_df[self.quantile_feature_colname] = processed_df.apply(_predict_single, axis=1)
        return processed_df

In [ ]:
# # Usage example:
# data = pd.DataFrame({
#     'category1': ["a"]*10 + ["b"]*10,
#     'category2': list(np.arange(10)) + list(np.arange(10,20))
# })
# # data["category2"]
# data

# qbc = QuantileBinClassifier(target_column="category2", group_by_cols=["category1"], n_bins=4)
# qbc.fit(data)

# bin_labels = qbc.predict(data)
# print(bin_labels)


In [ ]:
from sklearn.base import BaseEstimator, OneToOneFeatureMixin
from enum import StrEnum

DEFAULT_RANGE = np.array([0.0, 30.0])
TO_ADD_FOR_QUANTILE_EQUALITY = 15.0

class QuantileLevels(StrEnum):
    LOW = "low"
    HIGH = "high"

QUANTILE_VALUES = {
    QuantileLevels.LOW: 0.1,
    QuantileLevels.HIGH: 0.9
}


def compute_quantiles(group: pd.Series) -> pd.Series:  
    # Compute quantiles
    quantiles = {key: group.quantile(value) for key, value in QUANTILE_VALUES.items()}

    # Check for equal quantiles and adjust if necessary
    if quantiles[QuantileLevels.LOW] == quantiles[QuantileLevels.HIGH]:
        quantiles[QuantileLevels.LOW] -= TO_ADD_FOR_QUANTILE_EQUALITY
        quantiles[QuantileLevels.HIGH] += TO_ADD_FOR_QUANTILE_EQUALITY

    return pd.Series(quantiles)

class MarkupOptSpaceEstimator(BaseEstimator, OneToOneFeatureMixin):

    def __init__(self) -> None:
        super().__init__()

        self.default_range = DEFAULT_RANGE
        self.group_by_cols = GROUP_BY_COLS

        
        self._fitted_data: pd.DataFrame
        self._seen_classes: list
        
    def fit(self, X: pd.DataFrame,y=None):
        fitted_data = X.groupby(self.group_by_cols)[MARKUP_COLUMN].apply(
            compute_quantiles, 
            ).unstack().round(2)
        
        self._fitted_data = fitted_data
        self._seen_classes = self._fitted_data.index.to_list()

        return self

    def predict(self, X: pd.DataFrame):
        indexes_to_retrieve = X.set_index(self.group_by_cols).index.unique()
        output = pd.DataFrame(
            index=indexes_to_retrieve, columns=self._fitted_data.columns
        )
        seen_indexes = list(set(indexes_to_retrieve).intersection(self._seen_classes))
        not_seen_indexes = list(set(indexes_to_retrieve).difference(self._seen_classes))
        output.loc[seen_indexes] = self._fitted_data.loc[seen_indexes]
        output.loc[not_seen_indexes] = self.default_range

        processed_data = X.join(output, on=self.group_by_cols)
        return processed_data

In [ ]:
from sklearn.pipeline import Pipeline


estimator = Pipeline([
           ("seller_price_bins", QuantileBinClassifier()),
            ("markup_estimator_model", MarkupOptSpaceEstimator())
        ]
    )
estimator.fit(df)

In [ ]:
optimization_space = estimator.predict(df.sample(1000))
optimization_space.head()

In [ ]:
(optimization_space["high"] - optimization_space["low"]).mean()

In [ ]:
import os
import joblib

MODELS_FILEPATH = "dev/markup_optimization_interval"

os.makedirs(MODELS_FILEPATH, exist_ok=True)

joblib.dump(estimator, f"{MODELS_FILEPATH}/estimator.pkl")